# 02 - Vector Retrieval mit FAISS

Implementierung des FAISS-basierten Vektor-Retrievals für das RAG-Benchmarking.

## Ziele:
- FAQ-Korpus in Vektoren umwandeln
- FAISS-Index aufbauen
- Retrieval-Funktion implementieren
- Erste Tests mit Testfragen

In [1]:
# Imports
import json
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer # add to poetry
import pickle
from pathlib import Path
import time


## Daten laden

In [2]:
# FAQ-Korpus laden
with open('../data/faq_korpus.json', 'r', encoding='utf-8') as f:
    faq_documents = json.load(f)

# Testfragen laden
test_questions = pd.read_csv('../data/fragenliste.csv')

print(f"{len(faq_documents)} FAQ-Dokumente geladen")
print(f"{len(test_questions)} Testfragen geladen")

15 FAQ-Dokumente geladen
12 Testfragen geladen


## Embedding-Modell initialisieren

Wir nutzen ein deutsches/multilinguales Sentence-Transformer-Modell für bessere Performance bei deutschen Texten.

In [3]:
# Embedding-Modell laden (multilingual für deutsche Texte)
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embedding_model = SentenceTransformer(model_name)

print(f"Embedding-Modell geladen: {model_name}")
print(f"Embedding-Dimension: {embedding_model.get_sentence_embedding_dimension()}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\lukas\PycharmProjects\rag-benchmark\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lukas\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding-Modell geladen: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Embedding-Dimension: 384


## Dokumente für Embedding vorbereiten

Wir kombinieren Frage und Antwort für bessere Retrieval-Performance.

In [4]:
# Texte für Embedding vorbereiten (Frage + Antwort kombiniert)
documents_for_embedding = []
document_metadata = []

for doc in faq_documents:
    # Kombiniere Frage und Antwort für besseres Retrieval
    combined_text = f"Frage: {doc['question']}\n\nAntwort: {doc['answer']}"
    documents_for_embedding.append(combined_text)

    # Metadata für spätere Zuordnung
    document_metadata.append({
        'id': doc['id'],
        'question': doc['question'],
        'answer': doc['answer'],
        'category': doc['category'],
        'keywords': doc['keywords']
    })

print(f"{len(documents_for_embedding)} Dokumente für Embedding vorbereitet")
print(f"\nBeispiel kombinierter Text:")
print(documents_for_embedding[0][:200] + "...")

15 Dokumente für Embedding vorbereitet

Beispiel kombinierter Text:
Frage: Was ist Retrieval-Augmented Generation (RAG)?

Antwort: RAG ist eine Technik, die Large Language Models mit externen Wissensquellen verbindet. Das Modell kann relevante Informationen aus einer ...


## Embeddings generieren

In [5]:
# Embeddings für alle Dokumente generieren
print("Generiere Embeddings...")
start_time = time.time()

document_embeddings = embedding_model.encode(
    documents_for_embedding,
    show_progress_bar=True,
    convert_to_numpy=True
)

end_time = time.time()
print(f"Embeddings generiert in {end_time - start_time:.2f} Sekunden")
print(f"Shape: {document_embeddings.shape}")
print(f"Dimension pro Dokument: {document_embeddings.shape[1]}")

Generiere Embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generiert in 0.32 Sekunden
Shape: (15, 384)
Dimension pro Dokument: 384


## FAISS-Index aufbauen

In [6]:
# FAISS-Index erstellen (Cosinus-Ähnlichkeit)
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product für Cosinus-Ähnlichkeit

# Embeddings normalisieren für Cosinus-Ähnlichkeit
normalized_embeddings = document_embeddings / np.linalg.norm(document_embeddings, axis=1, keepdims=True)

# Zum Index hinzufügen
index.add(normalized_embeddings.astype('float32'))

print(f"FAISS-Index erstellt")
print(f"Index-Typ: {type(index)}")
print(f"Anzahl Vektoren: {index.ntotal}")
print(f"Dimension: {index.d}")

FAISS-Index erstellt
Index-Typ: <class 'faiss.swigfaiss_avx2.IndexFlatIP'>
Anzahl Vektoren: 15
Dimension: 384


## Retrieval-Funktion implementieren

In [7]:
def vector_retrieval(query, top_k=3):
    """
    Führt Vektor-Retrieval für eine Anfrage durch

    Args:
        query (str): Die Suchanfrage
        top_k (int): Anzahl der zurückzugebenden Dokumente

    Returns:
        list: Liste der relevantesten Dokumente mit Scores
    """
    # Query-Embedding generieren
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)

    # Normalisieren für Cosinus-Ähnlichkeit
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    # Suche im Index
    scores, indices = index.search(query_embedding.astype('float32'), top_k)

    # Ergebnisse zusammenstellen
    results = []
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        if idx != -1:  # Gültiger Index
            result = {
                'rank': i + 1,
                'score': float(score),
                'document_id': document_metadata[idx]['id'],
                'question': document_metadata[idx]['question'],
                'answer': document_metadata[idx]['answer'],
                'category': document_metadata[idx]['category'],
                'keywords': document_metadata[idx]['keywords']
            }
            results.append(result)

    return results

print("Retrieval-Funktion implementiert")

Retrieval-Funktion implementiert


## Erste Tests mit Beispielabfragen

In [8]:
# Test 1: Einfache Frage zu RAG
print("=== TEST 1: Einfache RAG-Frage ===")
query1 = "Was ist RAG?"
results1 = vector_retrieval(query1, top_k=3)

print(f"Query: '{query1}'")
print("\nTop 3 Ergebnisse:")
for result in results1:
    print(f"\n{result['rank']}. Score: {result['score']:.3f}")
    print(f"   Frage: {result['question']}")
    print(f"   Kategorie: {result['category']}")
    print(f"   Antwort: {result['answer'][:100]}...")

=== TEST 1: Einfache RAG-Frage ===
Query: 'Was ist RAG?'

Top 3 Ergebnisse:

1. Score: 0.585
   Frage: Was ist Retrieval-Augmented Generation (RAG)?
   Kategorie: RAG Basics
   Antwort: RAG ist eine Technik, die Large Language Models mit externen Wissensquellen verbindet. Das Modell ka...

2. Score: 0.455
   Frage: Welche Rolle spielt LangChain in RAG-Systemen?
   Kategorie: Tools
   Antwort: LangChain bietet einen einheitlichen Framework für RAG-Implementierungen. Es abstrahiert Retriever, ...

3. Score: 0.419
   Frage: Welche Zukunftstrends gibt es bei RAG-Technologien?
   Kategorie: Future Trends
   Antwort: Trends umfassen Agentic RAG mit Tool-Usage, Multi-modal RAG für Bilder/Videos, und Adaptive Retrieva...


In [9]:
# Test 2: Technische Frage zu Vektoren
print("\n=== TEST 2: Technische Frage ===")
query2 = "Wie funktioniert Vektorsuche?"
results2 = vector_retrieval(query2, top_k=3)

print(f"Query: '{query2}'")
print("\nTop 3 Ergebnisse:")
for result in results2:
    print(f"\n{result['rank']}. Score: {result['score']:.3f}")
    print(f"   Frage: {result['question']}")
    print(f"   Kategorie: {result['category']}")
    print(f"   Keywords: {result['keywords']}")


=== TEST 2: Technische Frage ===
Query: 'Wie funktioniert Vektorsuche?'

Top 3 Ergebnisse:

1. Score: 0.793
   Frage: Wie funktioniert Vektorsuche in RAG-Systemen?
   Kategorie: Vector Retrieval
   Keywords: ['vector search', 'embeddings', 'FAISS', 'similarity']

2. Score: 0.391
   Frage: Was ist Retrieval-Augmented Generation (RAG)?
   Kategorie: RAG Basics
   Keywords: ['RAG', 'retrieval', 'language model', 'knowledge base']

3. Score: 0.381
   Frage: Welche Evaluationsmetriken gibt es für RAG-Systeme?
   Kategorie: Evaluation
   Keywords: ['BLEU', 'ROUGE', 'evaluation', 'metrics', 'GPT-judge']


In [10]:
# Test 3: Vergleichsfrage
print("\n=== TEST 3: Vergleichsfrage ===")
query3 = "Unterschied zwischen Vektor und Graph Retrieval"
results3 = vector_retrieval(query3, top_k=3)

print(f"Query: '{query3}'")
print("\nTop 3 Ergebnisse:")
for result in results3:
    print(f"\n{result['rank']}. Score: {result['score']:.3f}")
    print(f"   Frage: {result['question']}")
    print(f"   Kategorie: {result['category']}")


=== TEST 3: Vergleichsfrage ===
Query: 'Unterschied zwischen Vektor und Graph Retrieval'

Top 3 Ergebnisse:

1. Score: 0.725
   Frage: Was sind die Vorteile von Graph-basiertem Retrieval?
   Kategorie: Graph Retrieval

2. Score: 0.586
   Frage: Was ist der Unterschied zwischen Dense und Sparse Retrieval?
   Kategorie: Retrieval Methods

3. Score: 0.567
   Frage: Wie funktioniert Vektorsuche in RAG-Systemen?
   Kategorie: Vector Retrieval


## Systematischer Test mit allen Testfragen

In [11]:
# Alle Testfragen durchlaufen
print("=== SYSTEMATISCHER TEST ALLER TESTFRAGEN ===")

vector_results = []

for idx, row in test_questions.iterrows():
    query = row['question']
    question_id = row['id']
    difficulty = row['difficulty']

    # Retrieval durchführen
    results = vector_retrieval(query, top_k=3)

    # Bestes Ergebnis speichern
    if results:
        best_result = results[0]
        vector_results.append({
            'question_id': question_id,
            'query': query,
            'difficulty': difficulty,
            'retrieved_doc_id': best_result['document_id'],
            'retrieval_score': best_result['score'],
            'retrieved_question': best_result['question'],
            'retrieved_category': best_result['category']
        })

    print(f"{question_id}: {difficulty} - Score: {results[0]['score']:.3f}")

print(f"\n{len(vector_results)} Testfragen abgearbeitet")

=== SYSTEMATISCHER TEST ALLER TESTFRAGEN ===
q001: easy - Score: 0.272
q002: medium - Score: 0.702
q003: medium - Score: 0.701
q004: hard - Score: 0.594
q005: easy - Score: 0.688
q006: hard - Score: 0.808
q007: medium - Score: 0.764
q008: medium - Score: 0.708
q009: hard - Score: 0.641
q010: medium - Score: 0.638
q011: easy - Score: 0.697
q012: hard - Score: 0.522

12 Testfragen abgearbeitet


## Ergebnisse analysieren

In [12]:
# Ergebnisse in DataFrame konvertieren
vector_df = pd.DataFrame(vector_results)

print("=== VECTOR RETRIEVAL ANALYSE ===")
print(f"Anzahl Testfragen: {len(vector_df)}")
print(f"Durchschnittlicher Score: {vector_df['retrieval_score'].mean():.3f}")
print(f"Minimaler Score: {vector_df['retrieval_score'].min():.3f}")
print(f"Maximaler Score: {vector_df['retrieval_score'].max():.3f}")

# Analyse nach Schwierigkeit
print("\n=== SCORES NACH SCHWIERIGKEIT ===")
difficulty_stats = vector_df.groupby('difficulty')['retrieval_score'].agg(['count', 'mean', 'std']).round(3)
print(difficulty_stats)

# Top 5 und Bottom 5 Ergebnisse
print("\n=== TOP 5 RETRIEVAL ERGEBNISSE ===")
top_5 = vector_df.nlargest(5, 'retrieval_score')[['question_id', 'difficulty', 'retrieval_score', 'query']]
print(top_5.to_string(index=False))

print("\n=== SCHWÄCHSTE 5 RETRIEVAL ERGEBNISSE ===")
bottom_5 = vector_df.nsmallest(5, 'retrieval_score')[['question_id', 'difficulty', 'retrieval_score', 'query']]
print(bottom_5.to_string(index=False))

=== VECTOR RETRIEVAL ANALYSE ===
Anzahl Testfragen: 12
Durchschnittlicher Score: 0.645
Minimaler Score: 0.272
Maximaler Score: 0.808

=== SCORES NACH SCHWIERIGKEIT ===
            count   mean    std
difficulty                     
easy            3  0.552  0.243
hard            4  0.641  0.121
medium          5  0.703  0.045

=== TOP 5 RETRIEVAL ERGEBNISSE ===
question_id difficulty  retrieval_score                                                                query
       q006       hard         0.807947                              Wie implementiere ich Hybrid Retrieval?
       q007     medium         0.763958                            Welche Vector Database sollte ich wählen?
       q008     medium         0.708346              Was sind häufige Probleme bei RAG und wie löse ich sie?
       q002     medium         0.702070      Was sind die Unterschiede zwischen Vektor- und Graph-Retrieval?
       q003     medium         0.701064 Welche Evaluationsmetriken sollte ich für mein RAG-

## Ergebnisse speichern

In [13]:
# Vector Retrieval Ergebnisse speichern
vector_df.to_csv('../results/vector_retrieval_results.csv', index=False, encoding='utf-8')

# FAISS Index und Metadata speichern für spätere Nutzung
faiss.write_index(index, '../data/faiss_index.bin')

with open('../data/vector_metadata.pkl', 'wb') as f:
    pickle.dump({
        'document_metadata': document_metadata,
        'model_name': model_name,
        'embedding_dimension': dimension
    }, f)

print("Ergebnisse gespeichert:")
print("  - ../results/vector_retrieval_results.csv")
print("  - ../data/faiss_index.bin")
print("  - ../data/vector_metadata.pkl")

Ergebnisse gespeichert:
  - ../results/vector_retrieval_results.csv
  - ../data/faiss_index.bin
  - ../data/vector_metadata.pkl


In [14]:
print("\n=== VECTOR RETRIEVAL ABGESCHLOSSEN ===")
print("FAISS-Index aufgebaut und getestet")
print("12 Testfragen erfolgreich abgearbeitet")
print("Ergebnisse gespeichert")
print(f"Durchschnittlicher Retrieval-Score: {vector_df['retrieval_score'].mean():.3f}")



=== VECTOR RETRIEVAL ABGESCHLOSSEN ===
FAISS-Index aufgebaut und getestet
12 Testfragen erfolgreich abgearbeitet
Ergebnisse gespeichert
Durchschnittlicher Retrieval-Score: 0.645
